In [1]:
import glob
import json
import pandas as pd
import numpy as np
import os
from pprint import pprint as pp

In [2]:
# loading testing template
import sys
myargs = sys.argv
if '-f' in myargs:
    try: FILE_NAME = myargs[2]
    except Exception as e:
        print(e)
        FILE_NAME = 'template'
else:
    os.system("cp ./template.zip ./temp/")
    FILE_NAME = 'template'
# if testing on jupyter notebook
if FILE_NAME[0] == "/":
    os.system("cp ./template.zip ./temp/")
    FILE_NAME = 'template'

In [3]:
FILE_NAME = 'SPH'

In [4]:
os.system("rm -rf ./temp/template")
os.system("rm -rf ./temp/{}".format(FILE_NAME))
os.system("unzip temp/{}.zip -d temp/{}".format(FILE_NAME, FILE_NAME))
os.system("tree")

0

In [5]:
!tree

.
|-- df-to-csv-ent.ipynb
|-- df-to-csv-ent.py
|-- df-to-csv.ipynb
|-- df-to-csv.py
|-- nohup.out
|-- server.py
`-- temp
    |-- 2019-06-03-07-39-24-027f9e42
    |   |-- agent.json
    |   |-- intents
    |   |   |-- Default\ Fallback\ Intent.json
    |   |   |-- Default\ Welcome\ Intent.json
    |   |   `-- Default\ Welcome\ Intent_usersays_en.json
    |   `-- package.json
    |-- 2019-06-03-07-39-24-027f9e42.csv
    |-- 2019-06-03-07-42-10-2745b507
    |   |-- agent.json
    |   |-- intents
    |   |   |-- Default\ Fallback\ Intent.json
    |   |   |-- Default\ Welcome\ Intent.json
    |   |   `-- Default\ Welcome\ Intent_usersays_en.json
    |   `-- package.json
    |-- 2019-06-03-07-42-10-2745b507.csv
    |-- 2019-06-03-07-44-47-d170bd51
    |   |-- agent.json
    |   |-- intents
    |   |   |-- Default\ Fallback\ Intent.json
    |   |   |-- Default\ Welcome\ Intent.json
    |   |   `-- Default\ Welcome\ Intent_usersays_en.json
    |   `-- package.json
    |-- 2019-06-03-07-44-47-d

In [6]:
files = sorted([entity for entity in glob.glob('temp/{}/entities/*.json'.format(FILE_NAME, FILE_NAME))])
files

['temp/SPH/entities/Amount.json',
 'temp/SPH/entities/Amount_entries_en.json',
 'temp/SPH/entities/Country-synonym.json',
 'temp/SPH/entities/Country-synonym_entries_en.json',
 'temp/SPH/entities/Invoice-Number.json',
 'temp/SPH/entities/Invoice-Number_entries_en.json',
 'temp/SPH/entities/Ok-synonym.json',
 'temp/SPH/entities/Ok-synonym_entries_en.json',
 'temp/SPH/entities/SAP-ID.json',
 'temp/SPH/entities/SAP-ID_entries_en.json',
 'temp/SPH/entities/Up.json',
 'temp/SPH/entities/Up_entries_en.json',
 'temp/SPH/entities/agree.json',
 'temp/SPH/entities/agree_entries_en.json',
 'temp/SPH/entities/commit.json',
 'temp/SPH/entities/commit_entries_en.json']

In [7]:
# assume English only
entity_hold = "none"
entity_jsons = []
for file in files:
    entity_name = file.rpartition('.')[0]
    if not entity_name.startswith(entity_hold):
        entity_jsons.append([file])
        entity_hold = entity_name
    else:
        entity_jsons[-1].append(file)
entity_jsons

[['temp/SPH/entities/Amount.json', 'temp/SPH/entities/Amount_entries_en.json'],
 ['temp/SPH/entities/Country-synonym.json',
  'temp/SPH/entities/Country-synonym_entries_en.json'],
 ['temp/SPH/entities/Invoice-Number.json',
  'temp/SPH/entities/Invoice-Number_entries_en.json'],
 ['temp/SPH/entities/Ok-synonym.json',
  'temp/SPH/entities/Ok-synonym_entries_en.json'],
 ['temp/SPH/entities/SAP-ID.json', 'temp/SPH/entities/SAP-ID_entries_en.json'],
 ['temp/SPH/entities/Up.json', 'temp/SPH/entities/Up_entries_en.json'],
 ['temp/SPH/entities/agree.json', 'temp/SPH/entities/agree_entries_en.json'],
 ['temp/SPH/entities/commit.json', 'temp/SPH/entities/commit_entries_en.json']]

In [8]:
entities = []
for entity_json in entity_jsons:
#     try:
        entity = {}
        entity_info_json = entity_json[0]
        entries_jsons = entity_json[1:] # should be one only

        with open(entity_info_json, encoding="utf-8") as f:
            entity_info = json.load(f)
#         pp(entity_info)

        entity["ENTITY_NAME"] = entity_info["name"]
        entity["ENTRIES"] = []
        
        for entries_json in entries_jsons: # should be one or none
            with open(entries_json, encoding="utf-8") as f:
                entries_info = json.load(f)
#                 pp(entries_info)
            for entries in entries_info:
                entity["ENTRIES"].append({entries['value']:entries['synonyms']})
        
        entities.append(entity)

In [9]:
pp(entities)

[{'ENTITY_NAME': 'Amount',
  'ENTRIES': [{'180': ['180']}, {'@Amount': ['@Amoun']}]},
 {'ENTITY_NAME': 'Country-synonym',
  'ENTRIES': [{'Malaysia': ['Malaysia']},
              {'Singapore': ['SG', 'Singapore', 'Spore']},
              {'UK': ['UK']},
              {'United States': ['United States']}]},
 {'ENTITY_NAME': 'Invoice-Number',
  'ENTRIES': [{'12-345-6789': ['12-345-6789']}]},
 {'ENTITY_NAME': 'Ok-synonym',
  'ENTRIES': [{'ok': ['ok']}, {'okay': ['okay']}, {'okeh': ['okeh']}]},
 {'ENTITY_NAME': 'SAP-ID',
  'ENTRIES': [{'123456': ['123456']},
              {'@Ok-synonym:Ok-synonym  @sys.number-integer:sys.number-integer': ['@Ok-synonym:Ok-synonym  '
                                                                                  '@sys.number-integer:sys.number-integer']}]},
 {'ENTITY_NAME': 'Up', 'ENTRIES': [{'@Up': ['@Upload']}]},
 {'ENTITY_NAME': 'agree', 'ENTRIES': [{'agree': ['agree']}]},
 {'ENTITY_NAME': 'commit',
  'ENTRIES': [{'no': ['commit', "i'm done", 'no', 'no m

In [10]:
row_list = []
for entity in entities:
    entity_name = entity["ENTITY_NAME"]
    for entries in entity["ENTRIES"]:
        row = [entity_name]
        for key, value in entries.items():
            row += [key]
            row += value
            entity_name = ""
            break
        row_list.append(row)

In [11]:
max_len = max([3] + [len(row) for row in row_list])

columns = ["ENTITY_NAME", "ENTITY_VALUE", "SYNONYMS"] + [""]*(max_len-3)
for i,row in enumerate(row_list):
    row_list[i] = row + [""]*(max_len-len(row))

In [12]:
df = pd.DataFrame(row_list, columns=columns)
df

,ENTITY_NAME,ENTITY_VALUE,SYNONYMS,,,
0,Amount,180,180,,,
1,,@Amount,@Amoun,,,
2,Country-synonym,Malaysia,Malaysia,,,
3,,Singapore,SG,Singapore,Spore,
4,,UK,UK,,,
5,,United States,United States,,,
6,Invoice-Number,12-345-6789,12-345-6789,,,
7,Ok-synonym,ok,ok,,,
8,,okay,okay,,,
9,,okeh,okeh,,,


In [13]:
df.to_csv("temp/{}-ent.csv".format(FILE_NAME))

In [14]:
# convert this notebook into a python script
# and then remove the last few lines
# get_ipython().system('jupyter nbconvert --to script df-to-csv-ent.ipynb')
# os.system("ex -snc '$-11,$d|x' df-to-csv-ent.py")